In [1]:
from ciphers.gpu import present80_gpu
from ciphers.cpu import present80 as present80_cpu

from ciphers.cpu import speck3264 as speck3264_cpu
from ciphers.gpu import speck3264_gpu

Computed: 0x5579c1387b228445
Expected: 0x5579c1387b228445


## Test for pure encryption

### Present

In [ ]:
# present80_cpu.benchmark_present80_numpy(batch_size=2000000)

In [ ]:
# present80_gpu.benchmark_gpu(batch_size=2000000)

### Speck3264

In [3]:
speck3264_cpu.check_testvectors()

In [3]:
speck3264_gpu.check_testvectors()

Final ciphertext (bit-level path): ['0xa868', '0x42f2']
Match expected ciphertext [0xa868, 0x42f2]?
True


In [9]:
speck3264_cpu.benchmark_numpy(batch_size=10_000_000)

10000000 blocks, 22 rounds: 18.5808s, tốc độ = 538,190.91 blocks/sec


In [8]:
speck3264_gpu.benchmark_gpu(batch_size=10_000_000)

10000000 blocks in 6.7190s, speed = 1488325.56 blocks/sec.


## Test for data generator (encryption x2 -> XORing)

### simple make_train_data (C1,C2)

#### CPU - PRESENT

In [ ]:
from generator import cipherpair_generator_cpu as cc

In [ ]:
cipher_name = "present" 
encryption_function = present80_cpu.encrypt
plain_bits = present80_cpu.plain_bits
key_bits = present80_cpu.key_bits
num_samples = 1_000_000
nr = 32

delta = cc.integer_to_binary_array(0x90000000000, plain_bits)
delta_key = 0
delta_plain = delta[:plain_bits]

data_generator = lambda num_samples, nr: cc.make_train_data(
    encryption_function, plain_bits, key_bits, num_samples, nr, delta_plain, delta_key
)

# maybe can put this func into \test ?
C_gpu, Y_gpu = cc.benchmark_make_train_data_cpu(encryption_function, num_samples, nr, plain_bits, key_bits)


[CPU/NumPy] Tạo 1000000 mẫu: 99.1131s (10,089.48 mẫu/giây)


In [ ]:
# maybe can put these func into \utils ?
cc.save_ciphertexpair_dataset_npz(C_gpu,Y_gpu,cipher_name=cipher_name,num_rounds=nr,num_samples=num_samples)

✅ Saved present dataset to ../../data\present_ciphertextpair_present_round32_samples1000000.npz


### GPU PRESENT

In [ ]:
from generator import cipherpair_generator_gpu as cp

In [ ]:
cipher_name = "present" 
encryption_function = present80_gpu.encrypt
plain_bits = present80_gpu.plain_bits
key_bits = present80_gpu.key_bits
num_samples = 1_000_000
nr = 32

delta = cp.integer_to_binary_array(0x90000000000, plain_bits)
delta_key = 0
delta_plain = delta[:plain_bits]

data_generator = lambda num_samples, nr: cp.make_train_data(
    encryption_function, plain_bits, key_bits, num_samples, nr, delta_plain, delta_key
)

# maybe can put this func into \test ?
C_gpu, Y_gpu = cp.benchmark_make_train_data_gpu(encryption_function, num_samples, nr, plain_bits,key_bits)


[GPU/CuPy] Tạo 1000000 mẫu: 6.6792s (149,718.28 mẫu/giây)


In [ ]:
# maybe can put these func into \utils ?
cc.save_ciphertexpair_dataset_npz(C_gpu,Y_gpu,cipher_name=cipher_name+"_gpu",num_rounds=nr,num_samples=num_samples)

✅ Saved present_gpu dataset to ../../data\present_gpu_ciphertextpair_present_gpu_round32_samples1000000.npz


## Test training (one round)

### DbitNet

In [ ]:
import tensorflow as tf
from models import dbitnet

PRESENT GPU

In [ ]:
from models import train_nets as tn

In [ ]:
input_size = plain_bits
model = dbitnet.make_model(2 * input_size)
optimizer = tf.keras.optimizers.Adam(amsgrad=True)

ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.ops`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
